1. 앞서 배운 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현합니다.
2. 이 체인은 영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답해야 합니다. (예: "탑건" -> "🛩️👨‍✈️🔥". "대부" -> "👨‍👨‍👦🔫🍝").
3. 항상 세 개의 이모티콘으로 답장하도록 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하여 체인에 예시를 제공하세요.
4. 메모리가 작동하는지 확인하려면 체인에 두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청하세요.

영화 제목을 질문하는 채팅 기능 > 세개의 이모티콘 형태 답장 기능 > 영화 제목과 이모티콘을 저장 기능 > 저장된 정보를 프롬프트에 전달 기능

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough

In [2]:
llm = ChatOpenAI(temperature=0.1)

In [3]:
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=300,
    return_messages=True,
)

In [4]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an emoji transformer that changes movie titles into 3 emoji icons.If the question is not the movie title, select a movie title that best matches the question and print the image. "),
    MessagesPlaceholder(variable_name="history"), 
    ("human", "{question}"),
])

In [5]:
def load_memory(_):
    return memory.load_memory_variables({})["history"]

In [6]:
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

In [ ]:
def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(햣
        {"input": question},
        {"output": result.content},
    )
    
    print(question, result) 

In [15]:
invoke_chain("대부")

대부 content='🕵️\u200d♂️🔫💰'


In [16]:
invoke_chain("겨울왕국")

겨울왕국 content='❄️👸🏼⛄'


In [17]:
invoke_chain("포레스트 검프")

포레스트 검프 content='🌲🏃🏼\u200d♂️🏌🏼\u200d♂️'


In [18]:
invoke_chain("제일 먼저 변경해 달라고 했던 영화 제목이 뭐였지?")

제일 먼저 변경해 달라고 했던 영화 제목이 뭐였지? content='🔄🎥❓'
